# Up and Running with Python3
# Data Analysis

In this notebook we will see:
- _Pandas_ for expanding data slicing capabilities
- _uproot_ for opening a ROOT file in pure python and numpy
- _Matplotlib_ for plotting (static)
- _Plotly_ for plotting (interactive)

## Pandas

Pandas is built on top of NumPy and it could be loosely described as NumPy with labels. That is, it is a package that deals with data in tabular form but which attaches more general labels and not just numerical indices to the rows and columns. 

Here, when we say, "tables," we include also one dimensional vectors, three dimensional data cubes, and so on. 

Pandas significantly enhances NumPy. In addition to adding data labels and descriptive indices, it is also more robust in handling common data formats and missing data. It also adds relational-database operations, such as joins. 

 Pandas DataFrames extend NumPy two-dimensional arrays by giving labels to the columns and if you provide an explicit index, also to the rows.
 
<img src="data/logos/1920px-Pandas_logo.svg.png" style="width: 250px;">

In [ ]:
import pandas as pd

A DataFrame is a table. It contains an array of individual entries, each of which has a certain value. Each entry corresponds to a row (or record) and a column.

For example, we can construct the record for a neutrino interaction:

In [ ]:
df = pd.DataFrame({'Particle': [14, 13, 2212], 
                   'Energy': [0.589766 , 0.489765, 0.959907], 
                   'State' : ['Initial', 'Final', 'Final']})
df

We can ask for the `shape` of the dataframe:

In [ ]:
df.shape

And for the lenght:

In [ ]:
len(df)

In Python, we can access the property of an object by accessing it as an attribute. A book object, for example, might have a title property, which we can access by calling book.title. Columns in a pandas DataFrame work in much the same way.

In this example, you can use df.Particle, for example

In [ ]:
# EXCERCISE: Try to access data from the DataFrame


If we have a Python dictionary, we can access its values using the indexing (`[]`) operator. We can do the same with columns in a DataFrame:

In [ ]:
# EXCERCISE: Try to access data from the DataFrame as if it were a dictionary


Pandas has some nice functions that can be applied to the dataframe, for example:

In [ ]:
print('mean energy:', df['Energy'].mean())
print('counts:', df['Energy'].count())

There are many more things to show about Pandas, and we only cover a few of them. You can find many more information online, for example in this [Kaggle tutorial](https://www.kaggle.com/learn/pandas).

## Uproot

[uproot](https://uproot.readthedocs.io/en/latest/) allows you to do ROOT I/O in pure Python and Numpy.

From the documentation page:

    uproot is a reader and a writer of the ROOT file format using only Python and Numpy. Unlike the standard C++ ROOT implementation, uproot is only an I/O library, primarily intended to stream data into machine learning libraries in Python. Unlike PyROOT and root_numpy, uproot does not depend on C++ ROOT. Instead, it uses Numpy to cast blocks of data from the ROOT file as Numpy arrays.
    
<img src="data/logos/uproot-logo-300px.png" style="width: 250px;">
   
Let's import it:

In [ ]:
import uproot

You can use `uproot.open(filename)` to open a new file.

- `.keys()` allows you to browse what is inside the file

- If you want to read TTree branch called, for example, 'particles', you do f

For example
```
f = uproot.open(filename.root)

# Browse what is inside the file
print('Keys:', f.keys())

# If you want to read a TTree called, for example, 'mytree', you do:
t = f['mytree']

# You can now print the name, title and number of entries in the tree
print('Tree name:', t.name)
print('Tree title:', t.title)
print('Number of entries:', t.numentries)
```

## Opening a CAF File with uproot

As an example on how we can use `uproot`, we are going to open an expore one of the CAFAna files that Fernanda showed in the tutorial yesterday!

The file is in the `data` directory, and is called `larout.caf.root`.

In [ ]:
# EXERCISE: Explore what is in the CAF file
f = uproot.open('data/larout.caf.root')

t = f['recTree']


We can use `.keys()` again to list what are the branches in the Tree.
We can also use `.show()` for a more in-depth look:

In [ ]:
t.show()

We are going to start loading all branches that start with `hdr`. This is the "header" and contains the metadata of the events.

We are also now converting the Tree to a `pandas` DataFrame:

In [ ]:
df_hdr = t.pandas.df('hdr*')
df_hdr

Now let's get all the slice information. They are in branches that start with `slc`.

In [ ]:
df_slc = f['recTree'].pandas.df('slc*', flatten=False)
df_slc

## A note about DataFrames:

- DataFrames are great but some care is required when designing their structure.

- DataFrames work great with a hierarchical structure. For example, if you store all the events and all the tracks per event this is fine, but problems appear if you start storing both tracks and showers in the same dataframe for example, as one event has different numbers of tracks and shower. Some of the problems that arise are solvable, but require some efforts. 

- Always make sure you design your DataFrame to be as flat as possible, and hirerchical.

Example, try to import both the `slc` and the `reco` branches in the same DataFrame:

In [ ]:
# f['recTree'].pandas.df(['slc*', 'reco*'] , flatten=False)

... yes... it fails 

## Matplotlib

Matplotlib is a plotting library for Python and NumPy. We will see more about matplotlib later, but here you can immediately see how easy it is to use from a pandas dataframe.

<img src="data/logos/matplotlib.png" style="width: 250px;">

From a pandas dataframe, we can immediately plot:

In [ ]:
df_slc['slc.vertex.x'].plot.hist()

In [ ]:
df_slc.plot.scatter(x='slc.vertex.z', y='slc.vertex.y', color='DarkBlue', label='Vertices');

## Seaborn

[seaborn](https://seaborn.pydata.org) is a Python data visualization library based on matplotlib.

It provides a high-level interface for drawing attractive and informative statistical graphics.

Let's see seaborn in action. First, let's import it:

In [ ]:
import seaborn as sns

Now, let do the same scatte plot, but with seaborn's `jointpolt` function:

In [ ]:
sns.jointplot('slc.vertex.z','slc.vertex.y', df_slc)

With seaborn, we can also plot more variables simultaneously, and look at their correlations.

Here, let's look at the correlation between:
- the number of points in a track
- the track lenght
- the $\cos\theta$ of the track

In [ ]:
variables = ['reco.trk.npts', 'reco.trk.len', 'reco.trk.costh']

df_reco = f['recTree'].pandas.df(variables, flatten=True)

sns.pairplot(df_reco, vars=variables, plot_kws={'s':6, 'alpha':1})

## Example 1: Higgs Potential with Matplotlib

Before we saw matplotlib in action calling it from a dataframe. Here we make a plot calling matplotlib ourself.

Let's make a plot of the Higgs potential
$V(\phi) = \mu^2\phi^\dagger\phi + \lambda(\phi^\dagger\phi)^2$
using Matplotlib. 

In this example we use pure NumPy and Matplotlib.

In [ ]:
import numpy as np

# Let's use a mu = -10 
# and a lambda of 1.4
mu = -10 # Higgs potential
#mu = 8  # Unique minimum
l = 1.4 

# Start in radial coordinates, 
# we generate a meshgrid with 
# r and theta values:
r = np.linspace(0, 3, 400)
theta = np.linspace(-0.9 * np.pi, 0.7 * np.pi, 200)
r, theta = np.meshgrid(r, theta)

# Then we go from radial
# to Cartesian coordinates
X = r * np.sin(theta)
Y = r * np.cos(theta)

# And we calculate the value of
# the potential:
V = mu*r**2 + l*r**4

Let's plot it now:

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8,6))

ax = plt.axes(projection='3d')

ax.plot_surface(X, Y, V, rstride=1, cstride=1,
                cmap='viridis', edgecolor='none', antialiased=False);

ax.set_xlabel('Re $\phi$', fontsize=18)
ax.set_ylabel('Im $\phi$', fontsize=18)
ax.set_zlabel('V($\phi)$', fontsize=18)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])

plt.show()

## Example 2: Pandas, Event Selection, and Plotting

In this example we take a ROOT file obtained from a LArSoft analyzer, which contains some truth basic variables. 

We use uproot to open the file and convert the `TTree` to a Pandas `DataFrame`.

We then apply a simple event selection using the Pandas `DataFrame::query()` function.

Finally, we make a plot of neutrino interactions as a function of neutrino energy.

Let's start with opening the file:

In [ ]:
f = uproot.open('data/sbnd_extracted_genie.root')
df = f['extractor/tree'].pandas.df()
df.keys()

In [ ]:
# EXERCISE: Select only event with neutrinos in the TPC x in [-200, 200], y in [-200, 200] and z in [0, 500]
# Also select only events with muon neutrinos interacting CC,


In [ ]:
# EXCERCISE: Make an histogram with the neutrino energy using "ax.hist"

import matplotlib.pyplot as plt

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,6))

#
# ...
#

plt.show()

Next, we see how to plot the same histogram using `step`:

In [ ]:
# First, define the bins using "np.linspace":
x_min = 0
x_max = 6
n_bins = 100
bins = np.linspace(x_min, x_max, n_bins+1)

# Get the data in numpy array:
data = df['nu_e'].values

# We use "np.histogram" to build the histogram:
h, _ = np.histogram(data, bins=bins)
h = np.append(h, h[-1])

# Finally, plot it!
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,6))

ax.step(bins, h)

plt.show()

## Plotly

Plotly provides online graphing, analytics, and statistics tools for many languages like Python, R, MATLAB, Perl, Julia, Arduino, and REST.

It allows to make nice interactive plots, like the ones we are going to do below.

Also, it has a nice a extention to make plots that remain interactive on the browser, called `Dash`.

<img src="data/logos/plotly.png" style="width: 250px;">

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="nu_e", nbins=100, histnorm='probability density')
fig.show()

In [ ]:
# EXCERCISE: Try to add option  marginal="violin", or "rug", "box"


In [ ]:
import yaml
import numpy as np
pmt_geom = yaml.load(open('data/sbnd_pmts_tpc0.yaml').read(), Loader=yaml.Loader)['PMTGeom']

pmts = {}
for key, value in pmt_geom.items():
    pmts[key] = value
    
pmts_y = []
pmts_z = []
for p in pmts.values():
    pmts_y.append(p[1])
    pmts_z.append(p[2])
    
pmt_mask = np.zeros(492, dtype=bool)
for i in range(0, 492):
    if i in pmt_geom.keys():
        pmt_mask[i] = True

In [ ]:
f = uproot.open('data/flashfinder_tree.root')
f.keys()
df = f['flashana/FlashTree'].pandas.df('flash*', flatten=False)
df.keys()

In [ ]:
event = 0
flash_pe_v = df['flash_pe_v'][event]

flash_pe_v = flash_pe_v[pmt_mask]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=pmts_z,
                                y=pmts_y,
                                mode='markers',
                                marker=dict(size=[30]*len(pmts_z),
                                            color=flash_pe_v),
                                text=flash_pe_v
                               )
               )

fig.show()

See [vis_icarus](http://web.stanford.edu/~kterao/Event144.html) by Kazu for a beautiful 3D rendering! 

## Fitting

Now we are going to look into a fitting example using Python.

For this example, we will take a ROOT file created by the SBND analyzer called `HitDumper`, which is the analyzer we are using for the SBND commissioning studies.

Among other information, this file contains all the reconstructed hits in the event.

The file is generated from particle gun, by producing muon that cross the detector longitudinally at different  `x` positions.

The idea of this example is:

- Get all the hits from TPC 0 and on the collection plane
- Group the hits in different `x` regions, based on the hit time
- Calculate the median hit integral value in every `x` region
- Fit these median values as a function of `x` (or drift time `t`) using $f(t) = e^{-t/\tau}$
- Extract the electron lifetime $\tau$

We are going to "slice" the SBND TPC in 10 regions of `x`:
<img src="data/sbnd_sliced.png" style="width: 300px;">
These regions can be identified by the CRT. 

Let's start by opening the ROOT file `hitdumper_tree.root`.

In [ ]:
file = uproot.open("data/hitdumper_tree.root")
df_original = file['hitdumper/hitdumpertree'].pandas.df("hit_*")

In [ ]:
df_original

As you can see, in this dataframe with have both `entry` and `subentry`. This is what is called a _multiindex_ dataframe. Here, in every event we have multiple hits, indexed by `subentry`.

Multiindex dataframe are powerful, and more details can be found in the Pandas [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

We can use Pandas `dropna()` to drop all the rows with NaN:

In [ ]:
df = df_original.dropna()

Also, we select hits from TPC 0 and from the collection plane only:

In [ ]:
# EXCERICE: Select only hits in hit_tpc == 0 and hit_plane == 2:


Now let's plot all the hit charge vs the hit time.

We can use matplotlib `hist2d` to make a 2D histogram.

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 6))

limits = [[0, 2500], [300, 800]]

h = ax.hist2d(df_original['hit_peakT'], df_original['hit_charge'], bins=100, range=limits)
plt.colorbar(h[3], ax=ax)

ax.set_ylabel('Hit Charge [ADC]',fontsize=18)
ax.set_xlabel('Drift Time [ticks]',fontsize=18)
ax.set_title('Simulated lifetime = 3 ms', loc='right', fontsize=18)
ax.tick_params(labelsize=15)
ax.grid(True)

plt.show()

Now, let's divide the hits in 10 different regions along the drift direction.

For every "x slice" we want the median of all the hit's charge.

First, let's construct the bin edges along the drift direction and also the center of these bins:

In [ ]:
bin_edges = np.arange(0, 2750, 250)
bin_center = [bin_edges[i]+(bin_edges[i+1]-bin_edges[i])/2 for i in range(len(bin_edges)-1)]
print('Bin edges:', bin_edges)
print('Bin centers:', bin_center)

Now we calculate the median of the hit integral in every `x` bin. Where:
- `hit_peakT` is the hit peak time
- `hit_charge` is the hit integral

In [ ]:
charge_median = []

for i in range(0, len(bin_edges)-1):
    start = bin_edges[i]
    end = bin_edges[i+1]
    query = f'hit_peakT > {start} and hit_peakT < {end}'
    
    df_select = df.query(query)
    
    median = np.median(df_select['hit_charge'].values)
    charge_median.append(median)


bin_center = np.array(bin_center)
charge_median = np.array(charge_median)

print('Bin centers:', bin_center)
print('Charge medians:', charge_median)

Let's plot the points that we just calculated:

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 6))

ax.errorbar(x=bin_center, y=charge_median, marker='o', linestyle='', label='Hit Charge Median')

ax.legend(fontsize=18, loc='best')
ax.set_xlabel('Drift Time [ticks]',fontsize=18)
ax.set_ylabel('Hit Charge [ADC]',fontsize=18)
ax.set_title('Simulated lifetime = 3 ms', loc='right', fontsize=18)
ax.tick_params(labelsize=15)
ax.grid(True)

plt.show()

Now, we want to fit these points with an exponential, this is how we do it in Python:

In [ ]:
from scipy.optimize import curve_fit

def exp(x, a0, tau):
    '''
    This defines the exponential function 
    that depends on a0 and tau
    '''
    return a0 * np.exp(-x/tau) 

def fit(func, x, y, seed=(), fit_range=None, **kwargs):
    '''
    Call this to fit a function func on data x,y
    You can pass the initial seeds, and the range 
    to use for the fit.
    '''
    if fit_range is not None:
        sel = (fit_range[0] <= x) & (x < fit_range[1])
        x, y = x[sel], y[sel]
        
    vals, cov = curve_fit(func, x, y, seed, **kwargs)
    
    fitf = lambda x: func(x, *vals)
    
    errors = np.sqrt(np.diag(cov))
    
    return fitf, vals, errors

Let's run the fit now!

In [ ]:
seed = 650., 6e3
fitf, vals, errs = fit(func=exp, 
                       x=bin_center, 
                       y=charge_median, 
                       seed=seed, 
                       fit_range=(0, 2500))
print('Fit results:', vals)

And finally, let's plot the results!

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 8))

_sampling = 2 # The 2 us sampling

ax.errorbar(x=bin_center, y=charge_median, marker='o', linestyle='', label='Hit Charge Median')

x = np.arange(0, 2500)

legend = f'Fit: $f(x) = a\cdot exp(-t/\\tau)$ \n $a$ = {vals[0]:.2} $\pm$ {errs[0]:.2} \n $\\tau$ = {vals[1]*1e-3/_sampling:.3} $\pm$ {errs[1]*1e-3/_sampling:.1}'

plt.plot(x, fitf(x), 'r-', label=legend)

ax.legend(fontsize=18, loc='best')
ax.set_xlabel('Drift Time [ticks]',fontsize=18)
ax.set_ylabel('Hit Charge [ADC]',fontsize=18)
ax.set_title('Simulated lifetime = 3 ms', loc='right', fontsize=18)
ax.tick_params(labelsize=15)
ax.grid(True)

plt.show()

In [ ]:
print('Well done! 👏👏👏')

Yes...you can use Unicode in your string literals! 😁